<a href="https://colab.research.google.com/github/Fuenfgeld/ATeamDatenmanagementUndArchivierung/blob/branch_leo/IMECOS_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# initialize the database, see import statements in *implementation.py
import requests
exec(requests.get('https://raw.githubusercontent.com/Fuenfgeld/ATeamDatenmanagementUndArchivierung/main/source_database_implementation.py').text)

Mounted at /content/gdrive
Successfully Connected to SQLite
Successfully created tables in the database
Successfully inserted source data
Source database ready for work
  
      conn    -    is the name of the connection object
      cur     -    is the name of the cursor
      
      


In [ ]:
# Testabfrage
cur.execute("""SELECT *
               FROM tb_payer_transitions""")
testabfrage = cur.fetchall()
for row in testabfrage:
  print(row)

In [3]:
#zeige alle tabellen
master_table_type = "SELECT name FROM sqlite_master WHERE type='table';" 
cur.execute(master_table_type)
print(cur.fetchall())


[('sqlite_sequence',), ('tb_patients',), ('tb_organizations',), ('tb_providers',), ('tb_payers',), ('tb_payer_transitions',), ('tb_encounters',), ('tb_careplans',), ('tb_conditions',), ('tb_devices',), ('tb_disease',), ('tb_imaging_studies',), ('tb_immunizations',), ('tb_observations',), ('tb_procedures',), ('tb_supplies',), ('tb_medications',)]


In [4]:
#12
#SQL_Abfrage_patient_Id_unique
sql = "SELECT Id, count(Id) FROM tb_patients GROUP BY Id HAVING (COUNT(Id)>1)"
cur.execute(sql)
patient_Id_unique = cur.fetchall()
for row in patient_Id_unique:
  print(row)

In [41]:
#13
sql13 = """
SELECT
   MIN(calc.percentage_explainable),
   MAX(calc.percentage_explainable),
   AVG(calc.percentage_explainable) 
FROM
   (
SELECT
tb_patients.Id,
encounters_total_cost_sum.value,
tb_patients.healthcare_expenses,
(encounters_total_cost_sum.value/round(tb_patients.healthcare_expenses,2))*100 as percentage_explainable
FROM
tb_patients,
(
SELECT tb_encounters.patient_id,
tb_encounters.patient_dso,
round(sum(tb_encounters.total_claim_cost),2) as value
FROM
tb_encounters
GROUP BY
tb_encounters.patient_id,
tb_encounters.patient_dso
) encounters_total_cost_sum
WHERE
tb_patients.Id = encounters_total_cost_sum.patient_id AND
tb_patients.dataset_origin = encounters_total_cost_sum.patient_dso) calc
"""

In [26]:
#15
sql15 = """
SELECT
tb_encounters.patient_id,
tb_encounters.Id,
immunizations_sum.value,
procedures_sum.value,
medications_sum.value,
tb_encounters.base_encounter_cost,
immunizations_sum.value+procedures_sum.value+medications_sum.value+tb_encounters.base_encounter_cost AS sum_hypothesis,
tb_encounters.total_claim_cost as tb_encounters_total_claim,
((immunizations_sum.value+procedures_sum.value+medications_sum.value+tb_encounters.base_encounter_cost)/tb_encounters.total_claim_cost)*100 AS percentage_explainable
FROM
tb_encounters,
(
SELECT
tb_medications.patient_id,
tb_medications.patient_dso,
tb_medications.encounter_id,
tb_medications.encounter_dso,
SUM(tb_medications.totalcost) as value
FROM
tb_medications
GROUP BY
tb_medications.patient_id,
tb_medications.encounter_id
) medications_sum,
(
SELECT
tb_procedures.patient_id,
tb_procedures.patient_dso,
tb_procedures.encounter_id,
tb_procedures.encounter_dso,
SUM(tb_procedures.base_cost) as value
FROM
tb_procedures
GROUP BY
tb_procedures.patient_id,
tb_procedures.encounter_id
) procedures_sum,
(
SELECT
tb_immunizations.patient_id,
tb_immunizations.patient_dso,
tb_immunizations.encounter_id,
tb_immunizations.encounter_dso,
SUM(tb_immunizations.base_cost) as value
FROM
tb_immunizations
GROUP BY
tb_immunizations.patient_id,
tb_immunizations.encounter_id
) immunizations_sum
WHERE
tb_encounters.id = medications_sum.encounter_id AND
tb_encounters.dataset_origin = medications_sum.encounter_dso AND
tb_encounters.patient_id = medications_sum.patient_id AND
tb_encounters.patient_dso = medications_sum.patient_dso AND
tb_encounters.id = procedures_sum.encounter_id AND
tb_encounters.dataset_origin = procedures_sum.encounter_dso AND
tb_encounters.patient_id = procedures_sum.patient_id AND
tb_encounters.patient_dso = procedures_sum.patient_dso AND
tb_encounters.id = immunizations_sum.encounter_id AND
tb_encounters.dataset_origin = immunizations_sum.encounter_dso AND
tb_encounters.patient_id = immunizations_sum.patient_id AND
tb_encounters.patient_dso = immunizations_sum.patient_dso
"""

In [28]:
sql15_2 = """SELECT * FROM tb_encounters WHERE base_encounter_cost < total_claim_cost"""

In [42]:
cur.execute(sql13)
response13 = cur.fetchall()
for row in response13:
  print(row)

(0.03482496742012313, 100.0, 1.6548514247049861)
